In [1]:
import warnings
warnings.filterwarnings('ignore')
import gc

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
# print(os.listdir("../input"))

import lightgbm as lgb
from sklearn.model_selection import *
from sklearn.metrics import mean_squared_error, make_scorer
from scipy.stats import mode, skew, kurtosis, entropy
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt
import seaborn as sns

import dask.dataframe as dd
from dask.multiprocessing import get

from tqdm import tqdm, tqdm_notebook

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
cols = ['f190486d6', '58e2e02e6', 'eeb9cd3aa', '9fd594eec', '6eef030c1', '15ace8c9f', 
        'fb0f5dbfe', '58e056e12', '20aa07010', '024c577b9', 'd6bb78916', 'b43a7cfd5', 
        '58232a6fb', '1702b5bf0', '324921c7b', '62e59a501', '2ec5b290f', '241f0f867', 
        'fb49e4212', '66ace2992', 'f74e8f13d', '5c6487af1', '963a49cdc', '26fc93eb7', 
        '1931ccfdd', '703885424', '70feb1494', '491b9ee45', '23310aa6f', 'e176a204a', 
        '6619d81fc', '1db387535', 
        'fc99f9426',
        '91f701ba2', '0572565c2', '190db8488', 'adb64ff71', 'c47340d97', 'c5a231d81', '0ff32eb98'
       ]

In [3]:
train = pd.read_csv("train.csv").drop('target', axis=1)
test = pd.read_csv("test.csv")
# df = pd.concat([train, test], axis=0)

KeyError: 'target'

In [16]:
def get_matching_rows(data, features, extra_features=[]):
    f1 = features[:-1]
    f2 = features[1:]

    for ef in extra_features:
        f1 += ef[:-1]
        f2 += ef[1:]

    d1 = data[f1].apply(tuple, axis=1).apply(hash).to_frame().rename(columns={0: 'key'})
    d1['ID'] = data['ID']    
    d2 = data[f2].apply(tuple, axis=1).apply(hash).to_frame().rename(columns={0: 'key'})
    d2['ID'] = data['ID']
    d3 = d2[~d2.duplicated(['key'], keep=False)]
    d4 = d1[~d1.duplicated(['key'], keep=False)]
    d5 = d4.merge(d3, how='inner', on='key')

    d = d1.merge(d5, how='left', on='key')
    d.fillna(0, inplace=True)

    ordered_ids = list(d[['ID_x', 'ID_y']][d.ID_x != 0].apply(list, axis=1))
    return ordered_ids

def get_matching_cols(data, rows, extra_rows=[]):
    dataT = data.transpose()
    dataT.columns = dataT.loc['ID']
    
    f1 = rows[:-1]
    f2 = rows[1:]

    for ef in extra_rows:
        f1 += ef[:-1]
        f2 += ef[1:]

    d1 = dataT[f1].apply(tuple, axis=1).apply(hash).to_frame().rename(columns={0: 'key'})
    d1['ID'] = dataT.index   
    d2 = dataT[f2].apply(tuple, axis=1).apply(hash).to_frame().rename(columns={0: 'key'})
    d2['ID'] = dataT.index
    d3 = d2[~d2.duplicated(['key'], keep=False)]
    d4 = d1[~d1.duplicated(['key'], keep=False)]
    d5 = d4.merge(d3, how='inner', on='key')

    d = d1.merge(d5, how='left', on='key')
    d.fillna(0, inplace=True)

    ordered_ids = list(d[['ID_x', 'ID_y']][d.ID_x != 0].apply(list, axis=1))
    return ordered_ids
#run this iteratively until you have no more links. Then prune

def chain_pairs(ordered_items):
    ordered_chains = []
    links_found = 0
    for i_1, op_chain in enumerate(ordered_items.copy()[:]):
        if op_chain[0] != op_chain[1]:
            end_chain = op_chain[-1]
            for i_2, op in enumerate(ordered_items.copy()[:]):
                if (end_chain == op[0]):
                    links_found += 1
                    op_chain.extend(op[1:])
                    end_chain = op_chain[-1]
            ordered_chains.append(op_chain)
    return links_found, ordered_chains


def prune_chain(ordered_chain):    
    ordered_chain = sorted(ordered_chain, key=len, reverse=True)
    new_chain = []
    id_lookup = {}
    for oc in ordered_chain:
        id_already_in_chain = False
        for idd in oc:
            if idd in id_lookup:
                id_already_in_chain = True
            id_lookup[idd] = idd
        if not id_already_in_chain:
            new_chain.append(oc)
    return sorted(new_chain, key=len, reverse=True)

In [30]:
ord_rows = get_matching_rows(train, cols)
links_found, ordered_chains = chain_pairs(ord_rows)
ordered_chain = prune_chain(ordered_chains)
[len(x) for x in ordered_chain]

[20,
 13,
 12,
 11,
 9,
 9,
 9,
 9,
 9,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 8,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 6,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 5,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 4,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,
 3,


In [ ]:
for i in range(len(ordered_chain)):
    if not i:
        ord_cols = get_matching_cols(train, ordered_chain[i])
    else:
        ord_cols.extend(get_matching_cols(train, ordered_chain[i]))

links_found, ordered_chains = chain_pairs(ord_cols)
ordered_chain = prune_chain(ordered_chains)
[len(x) for x in ordered_chain]
# x = train.transpose()
# x

KeyboardInterrupt: 

In [ ]:
import gc
gc.collect()